In [1]:
import scrapy
import re
from scrapy.crawler import CrawlerProcess
class ESSpider(scrapy.Spider):
    name = "spider_39"
    start_urls = [
        'http://quotes.toscrape.com/',
    ]
    def parse(self, response):
     
        for div in response.xpath('//div[@class="quote"]'): #where to start before clicking on downward arrow
            yield {
                'body': div.xpath("./span[@class='text']/text()").extract_first(),
                'author': div.xpath("span/small[@class='author']/text()").extract(),
                'tags': div.xpath('div[@class="tags"]/a[@class="tag"]/text()').extract()
            }
            
        # Get the URL of the previous page.
        next_page = response.xpath("/nav/ul[@class='pager'/li[@class='previous']/a/@href").extract_first()
        #/nav/ul[@class='pager'/li[@class='next']/a/@href
        pagenum = int(re.findall(r'\d+',next_page)[0])
        
        if next_page is not None and pagenum < 10:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

         
process = CrawlerProcess({
    'FEED_FORMAT': 'json',         
    'FEED_URI': 'trial_39.json',       
    'LOG_ENABLED': False,          
    'ROBOTSTXT_OBEY': True,
    'USER_AGENT': 'ThinkfulDataScienceBootcampCrawler (thinkful.com)', #not sure what this does?
    'AUTOTHROTTLE_ENABLED': True,
    'HTTPCACHE_ENABLED': True
})

process.crawl(ESSpider)
process.start()
print('Success!')

Success!


In [2]:
import pandas as pd
#pandas.read_json(open("ut1.json", "r", ))
all_pages = pd.read_json('trial_39.json', lines=False, orient='records')
print(all_pages.shape)
print(all_pages.head(50))

(10, 3)
                author                                               body  \
0    [Albert Einstein]  “The world as we have created it is a process ...   
1       [J.K. Rowling]  “It is our choices, Harry, that show what we t...   
2    [Albert Einstein]  “There are only two ways to live your life. On...   
3        [Jane Austen]  “The person, be it gentleman or lady, who has ...   
4     [Marilyn Monroe]  “Imperfection is beauty, madness is genius and...   
5    [Albert Einstein]  “Try not to become a man of success. Rather be...   
6         [André Gide]  “It is better to be hated for what you are tha...   
7   [Thomas A. Edison]  “I have not failed. I've just found 10,000 way...   
8  [Eleanor Roosevelt]  “A woman is like a tea bag; you never know how...   
9       [Steve Martin]  “A day without sunshine is like, you know, nig...   

                                             tags  
0        [change, deep-thoughts, thinking, world]  
1                            [abilities,